# Churn Modeling – Snowflake Environment Setup


This notebook sets up the full Snowflake environment for your churn modeling demo project.

It will:
- Create a **database**, **schema**, and **warehouse**
- Create **roles** and **grant privileges**
- Create the **customer data table**
- Create a **stage** for data loading

 **Manual input required:** You must define your Snowflake username before running the setup.


In [ ]:
# ==========================================================
#  MANDATORY MANUAL INPUT
# Replace the value below with your actual Snowflake username
YOUR_USER = "<REPLACE_WITH_YOUR_SNOWFLAKE_USERNAME>"
# ==========================================================

from snowflake.snowpark import Session

# Automatically connects using the current Snowflake context (user, role, warehouse, etc.)
session = Session.builder.getOrCreate()

print("Connected to Snowflake as:", YOUR_USER)


## Step 1: Create Database, Schema, and Warehouse

This section creates:
- A database `churn_modeling_db`
- A schema `churn_modeling_schema`
- A compute warehouse `churn_warehouse`


In [ ]:
session.sql("""
CREATE DATABASE IF NOT EXISTS churn_modeling_db;
""").collect()

session.sql("""
CREATE SCHEMA IF NOT EXISTS churn_modeling_db.churn_modeling_schema;
""").collect()

session.sql("""
CREATE WAREHOUSE IF NOT EXISTS churn_warehouse
  WITH WAREHOUSE_SIZE = 'SMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE
  INITIALLY_SUSPENDED = TRUE;
""").collect()

print("Database, schema, and warehouse created.")


## Step 2: Create Roles and Assign Privileges

This step:
- Creates a `churn_role`
- Grants it to your user
- Gives the role access to the database, schema, and warehouse
- Optionally creates a warehouse admin role


In [ ]:
# Create and assign role
session.sql("CREATE ROLE IF NOT EXISTS churn_role;").collect()
session.sql(f"GRANT ROLE churn_role TO USER {YOUR_USER};").collect()

# Privileges for churn_role (run each grant separately)
grants = [
    "GRANT USAGE ON DATABASE churn_modeling_db TO ROLE churn_role;",
    "GRANT USAGE ON SCHEMA churn_modeling_db.churn_modeling_schema TO ROLE churn_role;",
    "GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA churn_modeling_db.churn_modeling_schema TO ROLE churn_role;",
    "GRANT USAGE, OPERATE ON WAREHOUSE churn_warehouse TO ROLE churn_role;"
]

for query in grants:
    session.sql(query).collect()

# Optional: Warehouse admin role
session.sql("CREATE ROLE IF NOT EXISTS warehouse_admin_role;").collect()
session.sql("GRANT ALL PRIVILEGES ON WAREHOUSE churn_warehouse TO ROLE warehouse_admin_role;").collect()

print(f"Role 'churn_role' created and assigned to {YOUR_USER}")


## Step 3: Create the Customer Data Table

This table stores the churn modeling dataset with customer demographics,
financials, and churn flag.


In [ ]:
session.sql("""
CREATE OR REPLACE TABLE churn_modeling_db.churn_modeling_schema.customer_data (
    ID INT,
    ClientID INT PRIMARY KEY,
    Surname STRING,
    CreditScore INT,
    Geography STRING,
    Gender STRING,
    Age INT,
    Grade INT,
    AccountBalance FLOAT,
    ProductCount INT,
    OwnsCreditCard INT,
    IsActive INT,
    SalaryEstimated FLOAT,
    Churned INT
);
""").collect()

print(" Table 'customer_data' created successfully with updated column names.")



## Step 4
: Create Stage for CSV Upload

We create a named stage for storing the dataset temporarily before loading it
into the `customer_data` table.


In [ ]:
session.sql("""
CREATE OR REPLACE STAGE churn_modeling_db.churn_modeling_schema.churn_stage
  FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER = 1);
""").collect()

print(" Stage 'churn_stage' created successfully.")


## 
Environment Setup Complete!

You now have:
- Database: `churn_modeling_db`
- Schema: `churn_modeling_schema`
- Warehouse: `churn_warehouse`
- Role: `churn_role`
- Table: `customer_data`
- Stage: `churn_stage`

In [ ]:
# Upload the CSV file to the Snowflake stage
session.file.put(
    "Customer_Churn_Analysis.csv",
    "@churn_modeling_db.churn_modeling_schema.churn_stage",
    auto_compress=False
)

# Load the data from the stage into the table
session.sql("""
COPY INTO churn_modeling_db.churn_modeling_schema.customer_data
FROM @churn_modeling_db.churn_modeling_schema.churn_stage/Customer_Churn_Analysis.csv
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER = 1);
""").collect()

print(" Data successfully loaded from 'Customer_Churn_Analysis.csv' into 'customer_data' table.")


# Check results from table

In [ ]:
# Import Snowpark
from snowflake.snowpark import Session

# Connect to the existing session automatically in Snowflake Notebook
session = Session.builder.getOrCreate()

# Switch context (optional but safe)
session.sql("USE DATABASE churn_modeling_db").collect()
session.sql("USE SCHEMA churn_modeling_schema").collect()

# Check total rows
row_count = session.sql("SELECT COUNT(*) AS total_rows FROM customer_data").collect()[0]["TOTAL_ROWS"]
print(f" Total rows loaded: {row_count}")

# Preview a few rows of data
print("\n🔹 Sample data:")
session.sql("SELECT * FROM customer_data LIMIT 10").show()

# Check column names and data types
print("\n🔹 Table schema:")
session.sql("DESC TABLE customer_data").show()

# Optionally: get distinct values or stats for quick validation
print("\n🔹 Gender distribution:")
session.sql("""
SELECT Gender, COUNT(*) AS count
FROM customer_data
GROUP BY Gender
ORDER BY count DESC
""").show()
